In [1]:
# Importaciones
import sys
sys.path.append('..')

import pandas as pd
import numpy as np

# Importar módulos del paquete product_development
from product_development.config import (
    FEATURES, TARGET, CATEGORICAL_VARS, RAW_DATA_DIR, 
    PIPELINE_FILE, PREDICTIONS_FILE, TRAIN_TEST_SPLIT_RATIO
)
from product_development.dataset import load_raw_data, prepare_dataset
from product_development.modeling.train import load_pipeline
from product_development.modeling.predict import (
    predict, prepare_inference_data, save_predictions, run_inference
)

2025-11-29 10:00:58.690 | INFO     | product_development.config:<module>:17 - PROJ_ROOT path is: C:\Users\fjgon\OneDrive - Universidad Galileo\Trimestre 8\product_development


In [2]:
# 1. Cargar pipeline entrenado
sales_pipeline = load_pipeline(PIPELINE_FILE)
print("Pipeline cargado correctamente.")

2025-11-29 10:00:59.545 | INFO     | product_development.modeling.train:load_pipeline:284 - Cargando pipeline desde C:\Users\fjgon\OneDrive - Universidad Galileo\Trimestre 8\product_development\models\sales_pipeline.pkl
Pipeline cargado correctamente.


In [3]:
# 2. Cargar y preparar datos para inferencia
raw_data = load_raw_data(RAW_DATA_DIR / "train.csv")
dataset = prepare_dataset(raw_data)

# Obtener el 20% final para test (mismo split que entrenamiento)
n_samples = len(dataset)
n_train = int(n_samples * TRAIN_TEST_SPLIT_RATIO)

test_df = dataset.iloc[n_train:].copy()

print(f"Total registros: {n_samples}")
print(f"Registros de test: {len(test_df)}")
test_df.head()

2025-11-29 10:00:59.562 | INFO     | product_development.dataset:load_raw_data:49 - Cargando datos crudos desde C:\Users\fjgon\OneDrive - Universidad Galileo\Trimestre 8\product_development\data\raw\train.csv
2025-11-29 10:00:59.901 | INFO     | product_development.dataset:load_raw_data:51 - Cargados 913000 registros
2025-11-29 10:00:59.901 | INFO     | product_development.dataset:add_temporal_features:69 - Agregando características temporales
2025-11-29 10:01:00.036 | INFO     | product_development.dataset:convert_categorical_types:92 - Convirtiendo columnas categóricas a tipo object
Total registros: 913000
Registros de test: 182600


,date,store,item,sales,year,month,day_of_week_name
730400,2013-01-01,1,41,6,2013,1,Tuesday
730401,2013-01-02,1,41,15,2013,1,Wednesday
730402,2013-01-03,1,41,5,2013,1,Thursday
730403,2013-01-04,1,41,9,2013,1,Friday
730404,2013-01-05,1,41,13,2013,1,Saturday


In [4]:
# 3. Preparar datos para inferencia
X_test = prepare_inference_data(test_df)
print(f"Features preparadas: {X_test.shape}")
X_test.head()

2025-11-29 10:01:00.131 | INFO     | product_development.modeling.predict:prepare_inference_data:67 - Preparando datos para inferencia
Features preparadas: (182600, 5)


,store,item,year,month,day_of_week_name
730400,1,41,2013,1,Tuesday
730401,1,41,2013,1,Wednesday
730402,1,41,2013,1,Thursday
730403,1,41,2013,1,Friday
730404,1,41,2013,1,Saturday


In [5]:
# 4. Generar predicciones
y_test_pred = predict(sales_pipeline, X_test)

print(f"Predicciones generadas: {len(y_test_pred)}")
print(f"Primeras 10 predicciones: {y_test_pred[:10]}")

2025-11-29 10:01:00.174 | INFO     | product_development.modeling.predict:predict:47 - Generando predicciones para 182600 muestras
2025-11-29 10:01:00.384 | INFO     | product_development.modeling.predict:predict:49 - Predicciones generadas exitosamente
Predicciones generadas: 182600
Primeras 10 predicciones: [27.269253 27.235146 29.151873 31.31193  32.956093 35.11257  23.533398
 27.269253 27.235146 29.151873]


c:\Users\fjgon\OneDrive - Universidad Galileo\Trimestre 8\product_development\notebooks\..\product_development\transformers.py:135: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[var] = X[var].fillna(self.fill_value).astype(str)
c:\Users\fjgon\OneDrive - Universidad Galileo\Trimestre 8\product_development\notebooks\..\product_development\transformers.py:135: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[var] = X[var].fillna(self.fill_value).astype(str)


In [6]:
# 5. Guardar predicciones
output_df = save_predictions(test_df, y_test_pred, PREDICTIONS_FILE)

print(f"Archivo guardado en: {PREDICTIONS_FILE}")
output_df.head()

2025-11-29 10:01:00.397 | INFO     | product_development.modeling.predict:save_predictions:115 - Guardando predicciones en C:\Users\fjgon\OneDrive - Universidad Galileo\Trimestre 8\product_development\data\processed\test_predictions.csv
2025-11-29 10:01:02.887 | INFO     | product_development.modeling.predict:save_predictions:123 - Predicciones guardadas: 182600 registros
Archivo guardado en: C:\Users\fjgon\OneDrive - Universidad Galileo\Trimestre 8\product_development\data\processed\test_predictions.csv


,date,store,item,sales,year,month,day_of_week_name,sales_pred
730400,2013-01-01,1,41,6,2013,1,Tuesday,27.269253
730401,2013-01-02,1,41,15,2013,1,Wednesday,27.235146
730402,2013-01-03,1,41,5,2013,1,Thursday,29.151873
730403,2013-01-04,1,41,9,2013,1,Friday,31.311930
730404,2013-01-05,1,41,13,2013,1,Saturday,32.956093
